# Player Impact Score v 0.2

## Old Formula

### SCORE Component
player_goals_refined_df['SCORE'] = (player_goals_refined_df['Goals'] * 0.75) + \
                                  (player_goals_refined_df['Assist1'] * 0.715) + \
                                  (player_goals_refined_df['Assist2'] * 0.555)

### SHOTS Component
player_goals_refined_df['SHOTS'] = (player_goals_refined_df['Shots_On_Net'] * 0.075) + \
                                  (player_goals_refined_df['Shots_Off_Net'] * 0.075) - \
                                  (player_goals_refined_df['Shots_Blocked'] * 0.075)

### FACEOFFS Component
player_goals_refined_df['FACEOFFS'] = (player_goals_refined_df['Faceoff_Wins'] * 0.01) - \
                                     (player_goals_refined_df['Faceoff_Losses'] * 0.01)
#### Fill NaN values in 'Faceoff_Wins' and 'Faceoff_Losses' with 0
player_goals_refined_df['FACEOFFS'].fillna(0, inplace=True)


### TEAM Component
player_goals_refined_df['TEAM'] = (player_goals_refined_df['Goals_For_Team'] * 0.15) - \
                                 (player_goals_refined_df['Goals_Against_Team'] * 0.15)



### PENALTIES Component -  Negative because it is a penalty
player_goals_refined_df['PENALTIES'] = (player_goals_refined_df['Penalties_Taken'] * pp_success_rate) * -1

#### Fill all Nan Values with 0
player_goals_refined_df.fillna(0, inplace=True)

## Calculate the overall game score
player_goals_refined_df['Game_Score'] = player_goals_refined_df['SCORE'] + \
                                       player_goals_refined_df['SHOTS'] + \
                                       player_goals_refined_df['FACEOFFS'] + \
                                       player_goals_refined_df['TEAM'] - \
                                       player_goals_refined_df['PENALTIES']

In [ ]:
# Notes - Ideas for Imporovement

# 1. SCORE Component (Goals, Assists)
# Current Approach: Goals (0.75), Assist1 (0.715), Assist2 (0.555).
# - ADD A SIMILAR WEIGHTING SYSTEM TO THE SHOTS COMPONENT
# - Will regular a deep sql query but I want to identify games with ahigh impact on the game - goals that tie a game of put a team +1.
# - I'm also thinking of giving a bonus to unnassisted goals - otherwise they are worth less tot hte teams overall score and the goal scorer should be rewarded for the individual effort

# - SHORTHANDED POINTS - Should be worth a bigt bonus as they are huge in the game

# Suggested Improvements:
# Situation-Specific Weighting: Increase the weight for goals and assists in high-pressure situations, such as late-game scenarios, during close games, or in key power play/penalty kill moments.
# Positional Consideration: Adjust weights based on the player's position. For example, goals by defensemen could carry a higher weight due to their relative rarity.

# 2. SHOTS Component (Shots On/Off Net, Blocked)
# Current Approach: Same weight (0.075) for all shot types.
# Adjust blocked shots, they shouldn't be a negative as they are but they should be less than shots on net
# - Take a look at the weight - find way to justify. maybe OnNet-100 offnet-75 blocked-25
# - Weight by Situation - CLOSE_GAME = 100, PP = 75, # Remaining EVEN Strength SHots = 50
# Suggested Improvements:

# Contextual Weighting: Adjust weights based on game situation, like shots during power plays or close_game - both are in data

# 3. FACEOFFS Component (Wins, Losses)
# Current Approach: Equal weight (0.01) for wins and losses.

# Suggested Improvements:
# Increase the current weight - add more weight to Close, PP and PK faceoffs if possible
# Don't have the ability to diff face off success by zone or game time (e.g., late in a close game)

# 4. TEAM Component (Goals For/Against While on Ice)
# - Just use the redily available plus/minus data and weigh it the old way - UNTIL something better is available


# # 5. PENALTIES Component (Penalties Taken)
# Current Approach: Weighted by the overall PP% for entire dataset

# Suggested Improvements:
# Weighted by the opposing team's power play success rate.
# Type of Penalty: Differentiate between minor and major penalties, as they have different impacts on the game.
# Timing of Penalty: Penalties taken in critical moments (e.g., late in a close game) could have a higher negative weight.
#  - IDEA: Maybe Disregard penalties when the other team takes one at the same time


